 ### Data Preparation
 #### Unstructured Data Analysis

In [1]:
import glob
import numpy as np
import pandas as pd
from ipynb.fs.full.uda_functions import makeWordList, read_docs

In [2]:
files   = [i for i in glob.glob('./data_sessions/*/*.txt')]
files71 = [i for i in glob.glob('./data_sessions/Session 71 - 2016/*')]
docs    = [read_docs(i) for i in files] + [read_docs(i) for i in files71]

In [3]:
year    = [int(i.split('\\')[2][7:-4].strip('_')) for i in files] + [int(i.split('\\')[1][-4:]) for i in files71]
country = [i.split('\\')[2][:3] for i in files] + [i.split('\\')[1][:3] for i in files71]

In [4]:
for i,j in enumerate(year):
    if j == 11: year[i] = 2011
    if j == 12: year[i] = 2012
    if j == 13: year[i] = 2013
    if j == 14: year[i] = 2014
    if j == 15: year[i] = 2015

In [5]:
undocs   = pd.DataFrame({'text': docs, 'year': year, 'countryID': country})
codebook = pd.read_csv('./data_sessions/speakers_by_session.csv', sep = ',', header = 0)
wbank    = pd.read_csv('./data_worldbank/ESGData.csv', sep = ',', header = 0)
wbankdet = pd.read_csv('./data_worldbank/ESGCountry_country_details.csv', sep = ',', header = 0)

In [6]:
codebook = codebook.rename(columns = {'Year':'year','ISO Code': 'countryID'})
wbank    = wbank.rename(columns = {"Country Code": "countryID", "Indicator Code": "indicator"})
wbankdet = wbankdet[['Country Code','Region','Income Group']]
wbankdet = wbankdet.rename(columns = {'Country Code':'countryID','Region':'region','Income Group':'income_level'})

In [7]:
wbank2   = pd.wide_to_long(wbank, ["y"], i = ["countryID","indicator"], j = "year").reset_index()
wbank2   = wbank2.rename(columns = {'y': 'indicator_val'}) 

In [15]:
db = pd.merge(undocs, codebook, on = ['year', 'countryID'])
db = pd.merge(db, wbank2, on = ['year', 'countryID'])
db = pd.merge(db, wbankdet, on = 'countryID')

In [16]:
db = db.rename(columns = {'Session':'session','Name of Person Speaking':'speaker','text':'statement',
                          'Language':'language','Notes':'notes','indicator':'indic_id',
                          'Indicator Name':'indic_name','Country Name':'country_name','indicator_val':'indic_val',
                          'Post':'speaker_post'})

In [18]:
db = db[['session','year','countryID','country_name','region','statement','speaker','speaker_post','indic_id','indic_name','indic_val','income_level','language']]

In [19]:
db.to_csv('db_final.txt')